In [0]:
! kill -9 -1

In [0]:
import os
from google.colab import drive
drive.mount('/gdrive')
os.symlink('/gdrive/My Drive', '/content/gdrive')
!ls -l /content/gdrive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
total 12
drwx------ 2 root root 4096 May 20 16:10 'All code and data for Yolo'
drwx------ 2 root root 4096 Jun 13 08:32  Energy
drwx------ 2 root root 4096 May 20 16:12 '中間產生檔案 or 舊檔'


In [0]:
import os
os.chdir('/gdrive/My Drive/All code and data for Yolo/Label and Pixel')
os.listdir()

['train_pixel_firsthalf_array.npy',
 'train_pixel_secondhalf_array.npy',
 'test_label_array_version2.npy',
 'train_label_array_version2.npy',
 'train_label_array.npy',
 'test_label_array.npy',
 'train_label_array_18*32.npy',
 'test_label_array_18*32.npy',
 'train_pixel0.5_array_new.npy',
 'train_pixel1_array_new.npy',
 'train_pixel1.5_array_new.npy',
 'train_pixel2_array_new.npy',
 'yolo_model_loss1_batch_size_40_epoch_15_1.json',
 'yolo_model_loss1_batch_size_40_epoch_15_1.h5']

In [0]:
%env KERAS_BACKEND=tensorflow
from keras.models import Sequential ,Model
from keras.layers import Dense, Activation, Flatten,Input
from keras.layers import Conv2D, MaxPooling2D, Reshape
from keras.optimizers import SGD, Adam
from keras.layers import LeakyReLU
import numpy as np
import tensorflow as tf

env: KERAS_BACKEND=tensorflow


In [0]:
xtrain1 = np.load("train_pixel0.5_array_new.npy")
#xtrain2 = np.load("train_pixel1_array_new.npy")
ytrain = np.load('train_label_array_18*32.npy')


ytrain.shape

(2880, 5, 18, 32)

In [0]:
ytrain=ytrain[0:360]

In [0]:
xtrain1.shape

(360, 720, 1280, 3)

In [0]:
def construct_Conv2D(f_num,f_size,stride=(1,1)):
    return model.add(Conv2D(f_num,f_size,strides=stride,padding="same"))

In [0]:
from keras.layers import BatchNormalization

In [0]:
IMAGE_H, IMAGE_W = 720, 1280
grid = 40
GRID_H,  GRID_W = 18, 32
NO_OBJECT_SCALE = 1.0
OBJECT_SCALE = 5.0
COORD_SCALE = 1.0
LOSS_noobj = 0.5
no_batch =  30

In [0]:
def custom_loss(y_true,y_pred):
    true = tf.equal(tf.ones([GRID_H,  GRID_W], tf.float32), 1)
    false = tf.equal(tf.ones([GRID_H,  GRID_W], tf.float32), 0)
    true_mat = tf.tile([tf.concat([tf.tile([true], [3, 1, 1]), tf.tile([false], [2, 1, 1])], 0)], [no_batch,1,1,1])
    xy_pred = tf.where(true_mat, tf.sigmoid(y_pred), tf.zeros([no_batch, 5, GRID_H, GRID_W],tf.float32))
    wh_pred = tf.where(true_mat, tf.zeros([no_batch, 5, GRID_H, GRID_W],tf.float32), tf.exp(y_pred))
    adjusted_y_pred = xy_pred+wh_pred
    true_2 = tf.equal(y_true[:,0], tf.ones([no_batch, GRID_H,  GRID_W], tf.float32))
    no_object_loss = tf.where(true_2, tf.zeros([no_batch,GRID_H,  GRID_W], tf.float32), LOSS_noobj*(y_true[:,0]-adjusted_y_pred[:,0])**2)
    object_loss = tf.where(true_2,(y_true[:,0]-adjusted_y_pred[:,0])**2
                           +OBJECT_SCALE*(y_true[:,1]-adjusted_y_pred[:,1])**2
                           +OBJECT_SCALE*(y_true[:,2]-adjusted_y_pred[:,2])**2
                           +OBJECT_SCALE*(y_true[:,3]**0.5-adjusted_y_pred[:,3]**0.5)**2
                           +OBJECT_SCALE*(y_true[:,4]**0.5-adjusted_y_pred[:,4]**0.5)**2
                           , tf.zeros([no_batch,GRID_H,  GRID_W], tf.float32))
    total_loss = no_object_loss + object_loss
    return total_loss
  

## 建立 model

In [0]:
input_img=(IMAGE_H , IMAGE_W, 3)

In [0]:
model=Sequential()
##Layer 1
model.add(Conv2D(64,(18,32),strides=(9,16),padding="same",input_shape=input_img))
model.add(BatchNormalization(name='norm_1'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),strides=2))

## Layer 2
construct_Conv2D(64,(3,3))
model.add(BatchNormalization(name='norm_2'))
model.add(LeakyReLU(alpha=0.1))
## model.add(MaxPooling2D(pool_size=(2, 2),strides=2))

## Layer 3
construct_Conv2D(128,(1,1))
model.add(BatchNormalization(name='norm_3'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(256,(3,3))
model.add(BatchNormalization(name='norm_4'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(256,(1,1))
model.add(BatchNormalization(name='norm_5'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(512,(3,3))
model.add(BatchNormalization(name='norm_6'))
model.add(LeakyReLU(alpha=0.1))
##model.add(MaxPooling2D(pool_size=(2, 2)))
##model.add(MaxPooling2D(pool_size=(2, 2),strides=2))

## Layer 4
construct_Conv2D(256,(1,1))
model.add(BatchNormalization(name='norm_7'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(512,(3,3))
model.add(BatchNormalization(name='norm_8'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(512,(1,1))
model.add(BatchNormalization(name='norm_9'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(512,(3,3))
model.add(BatchNormalization(name='norm_10'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
##----------------------------------- deeper
construct_Conv2D(256,(1,1))
model.add(BatchNormalization(name='norm_11'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(512,(3,3))
model.add(BatchNormalization(name='norm_12'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(256,(1,1))
model.add(BatchNormalization(name='norm_13'))
model.add(LeakyReLU(alpha=0.1))
##model.add(MaxPooling2D(pool_size=(2, 2)))

construct_Conv2D(512,(3,3))
model.add(BatchNormalization(name='norm_14'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(256,(1,1))
model.add(BatchNormalization(name='norm_15'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(512,(3,3))
model.add(BatchNormalization(name='norm_16'))
model.add(LeakyReLU(alpha=0.1))
##--------------------------------------
construct_Conv2D(512,(1,1))
model.add(BatchNormalization(name='norm_17'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(1024,(3,3))
model.add(BatchNormalization(name='norm_18'))
model.add(LeakyReLU(alpha=0.1))
##model.add(MaxPooling2D(pool_size=(2, 2)))
##model.add(MaxPooling2D(pool_size=(2, 2),strides=2))

## Layer 5
construct_Conv2D(512,(1,1))
model.add(BatchNormalization(name='norm_19'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(1024,(3,3))
model.add(BatchNormalization(name='norm_20'))
model.add(LeakyReLU(alpha=0.1))
##model.add(MaxPooling2D(pool_size=(2, 2)))
##-------------deeper
construct_Conv2D(512,(1,1))
model.add(BatchNormalization(name='norm_21'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(1024,(3,3))
model.add(BatchNormalization(name='norm_22'))
model.add(LeakyReLU(alpha=0.1))
##---------------------
construct_Conv2D(512,(3,3))
model.add(BatchNormalization(name='norm_23'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(1024,(3,3))## strides(2,2)
model.add(BatchNormalization(name='norm_24'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
## Layer 6
construct_Conv2D(1024,(3,3))
model.add(BatchNormalization(name='norm_25'))
model.add(LeakyReLU(alpha=0.1))

construct_Conv2D(1024,(3,3))
model.add(BatchNormalization(name='norm_26'))
model.add(LeakyReLU(alpha=0.1))

## Last two layer
model.add(Conv2D(256,(1,1),padding="same"))
model.add(BatchNormalization(name='norm_27'))
model.add(LeakyReLU(alpha=0.1))

model.add(Flatten())
model.add(Dense(2880,input_dim=25600))
model.add(Activation('linear'))

model.add(Reshape((5,18,32)))
model.compile(loss=custom_loss, optimizer=SGD(lr=0.001) )

In [0]:
model.summary()

In [0]:
model.fit(xtrain1,ytrain,batch_size=no_batch, epochs=2**9)

Epoch 1/512
360/360 [==============================] - 27s 74ms/step - loss: 0.1900
Epoch 2/512
360/360 [==============================] - 13s 36ms/step - loss: 0.1855
Epoch 3/512
360/360 [==============================] - 14s 38ms/step - loss: 0.1832
Epoch 4/512
360/360 [==============================] - 14s 38ms/step - loss: 0.1798
Epoch 5/512
360/360 [==============================] - 13s 37ms/step - loss: 0.1759
Epoch 6/512
360/360 [==============================] - 13s 37ms/step - loss: 0.1734
Epoch 7/512
360/360 [==============================] - 14s 38ms/step - loss: 0.1711
Epoch 8/512
360/360 [==============================] - 14s 39ms/step - loss: 0.1693
Epoch 9/512
360/360 [==============================] - 14s 38ms/step - loss: 0.1654
Epoch 10/512
360/360 [==============================] - 13s 37ms/step - loss: 0.1639
Epoch 11/512
360/360 [==============================] - 14s 38ms/step - loss: 0.1618
Epoch 12/512
360/360 [==============================] - 14s 39ms/step - lo

In [0]:
model_json= model.to_json()
open('yolo_model_loss1_batch_size_40_epoch_15_1.json','w').write(model_json)
model.save_weights('yolo_model_loss1_batch_size_40_epoch_15_1.h5')

## 繼續訓練

In [0]:
! kill -9 -1

In [0]:
import os
from google.colab import drive
drive.mount('/gdrive')
os.symlink('/gdrive/My Drive', '/content/gdrive')
!ls -l /content/gdrive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
total 12
drwx------ 2 root root 4096 May 20 16:10 'All code and data for Yolo'
drwx------ 2 root root 4096 Jun 13 08:32  Energy
drwx------ 2 root root 4096 May 20 16:12 '中間產生檔案 or 舊檔'


In [0]:
import os
os.chdir('/gdrive/My Drive/All code and data for Yolo/Label and Pixel')
os.listdir()

['train_pixel_firsthalf_array.npy',
 'train_pixel_secondhalf_array.npy',
 'test_label_array_version2.npy',
 'train_label_array_version2.npy',
 'train_label_array.npy',
 'test_label_array.npy',
 'train_label_array_18*32.npy',
 'test_label_array_18*32.npy',
 'train_pixel0.5_array_new.npy',
 'train_pixel1_array_new.npy',
 'train_pixel1.5_array_new.npy',
 'train_pixel2_array_new.npy',
 'yolo_model_loss1_batch_size_40_epoch_15_1.json',
 'yolo_model_loss1_batch_size_40_epoch_15_1.h5']

In [0]:
%env KERAS_BACKEND=tensorflow
from keras.models import Sequential ,Model
from keras.layers import Dense, Activation, Flatten,Input
from keras.layers import Conv2D, MaxPooling2D, Reshape
from keras.optimizers import SGD, Adam
from keras.layers import LeakyReLU
import numpy as np
import tensorflow as tf

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


In [0]:
xtrain1 = np.load("train_pixel1_array_new.npy")
ytrain = np.load('train_label_array_18*32.npy')


ytrain.shape

(2880, 5, 18, 32)

In [0]:
ytrain=ytrain[360:720]

In [0]:
xtrain1.shape

(360, 720, 1280, 3)

In [0]:
IMAGE_H, IMAGE_W = 720, 1280
grid = 40
GRID_H,  GRID_W = 18, 32
NO_OBJECT_SCALE = 1.0
OBJECT_SCALE = 5.0
COORD_SCALE = 1.0
LOSS_noobj = 0.5
no_batch =  30

In [0]:
def custom_loss(y_true,y_pred):
    true = tf.equal(tf.ones([GRID_H,  GRID_W], tf.float32), 1)
    false = tf.equal(tf.ones([GRID_H,  GRID_W], tf.float32), 0)
    true_mat = tf.tile([tf.concat([tf.tile([true], [3, 1, 1]), tf.tile([false], [2, 1, 1])], 0)], [no_batch,1,1,1])
    xy_pred = tf.where(true_mat, tf.sigmoid(y_pred), tf.zeros([no_batch, 5, GRID_H, GRID_W],tf.float32))
    wh_pred = tf.where(true_mat, tf.zeros([no_batch, 5, GRID_H, GRID_W],tf.float32), tf.exp(y_pred))
    adjusted_y_pred = xy_pred+wh_pred
    true_2 = tf.equal(y_true[:,0], tf.ones([no_batch, GRID_H,  GRID_W], tf.float32))
    no_object_loss = tf.where(true_2, tf.zeros([no_batch,GRID_H,  GRID_W], tf.float32), LOSS_noobj*(y_true[:,0]-adjusted_y_pred[:,0])**2)
    object_loss = tf.where(true_2,(y_true[:,0]-adjusted_y_pred[:,0])**2
                           +OBJECT_SCALE*(y_true[:,1]-adjusted_y_pred[:,1])**2
                           +OBJECT_SCALE*(y_true[:,2]-adjusted_y_pred[:,2])**2
                           +OBJECT_SCALE*(y_true[:,3]**0.5-adjusted_y_pred[:,3]**0.5)**2
                           +OBJECT_SCALE*(y_true[:,4]**0.5-adjusted_y_pred[:,4]**0.5)**2
                           , tf.zeros([no_batch,GRID_H,  GRID_W], tf.float32))
    total_loss = no_object_loss + object_loss
    return total_loss
  

In [0]:
from keras.models import model_from_json
model = model_from_json(open('yolo_model_loss1_batch_size_40_epoch_15_1.json').read())
model.load_weights('yolo_model_loss1_batch_size_40_epoch_15_1.h5')

W0804 05:24:48.983030 140595017000832 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0804 05:24:49.050933 140595017000832 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0804 05:24:49.090828 140595017000832 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0804 05:24:49.091689 140595017000832 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0804 05:24:49.092504 140595017

In [0]:
model.compile(loss=custom_loss, optimizer=SGD(lr=0.001) )
model.fit(xtrain1,ytrain,batch_size=no_batch, epochs=2**9)

W0804 05:25:35.789767 140595017000832 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0804 05:25:35.803823 140595017000832 deprecation.py:323] From <ipython-input-8-e2d6184a2baf>:5: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/512
360/360 [==============================] - 28s 77ms/step - loss: 0.0333
Epoch 2/512
360/360 [==============================] - 13s 36ms/step - loss: 0.0316
Epoch 3/512
360/360 [==============================] - 13s 36ms/step - loss: 0.0316
Epoch 4/512
360/360 [==============================] - 14s 38ms/step - loss: 0.0311
Epoch 5/512
360/360 [==============================] - 13s 37ms/step - loss: 0.0306
Epoch 6/512
360/360 [==============================] - 14s 38ms/step - loss: 0.0301
Epoch 7/512
360/360 [==============================] - 13s 36ms/step - loss: 0.0287
Epoch 8/512
360/360 [==============================] - 13s 36ms/step - loss: 0.0277
Epoch 9/512
360/360 [==============================] - 13s 35ms/step - loss: 0.0280
Epoch 10/512
360/360 [==============================] - 13s 36ms/step - loss: 0.0266
Epoch 11/512
360/360 [==============================] - 13s 37ms/step - loss: 0.0272
Epoch 12/512
360/360 [==============================] - 13s 36ms/step - lo

In [0]:
model_json= model.to_json()
open('yolo_model_loss1_batch_size_30_epoch_2**9_2.json','w').write(model_json)
model.save_weights('yolo_model_loss1_batch_size_30_epoch_2**9_2.h5')

## 新增 cross validation

In [0]:
! kill -9 -1

In [0]:
import os
from google.colab import drive
drive.mount('/gdrive')
os.symlink('/gdrive/My Drive', '/content/gdrive')
!ls -l /content/gdrive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
total 12
drwx------ 2 root root 4096 May 20 16:10 'All code and data for Yolo'
drwx------ 2 root root 4096 Jun 13 08:32  Energy
drwx------ 2 root root 4096 May 20 16:12 '中間產生檔案 or 舊檔'


In [0]:
import os
os.chdir('/gdrive/My Drive/All code and data for Yolo/Label and Pixel')
os.listdir()

['train_pixel_firsthalf_array.npy',
 'train_pixel_secondhalf_array.npy',
 'test_label_array_version2.npy',
 'train_label_array_version2.npy',
 'train_label_array.npy',
 'test_label_array.npy',
 'train_label_array_18*32.npy',
 'test_label_array_18*32.npy',
 'train_pixel0.5_array_new.npy',
 'train_pixel1_array_new.npy',
 'train_pixel1.5_array_new.npy',
 'train_pixel2_array_new.npy',
 'yolo_model_loss1_batch_size_40_epoch_15_1.json',
 'yolo_model_loss1_batch_size_40_epoch_15_1.h5',
 'yolo_model_loss1_batch_size_30_epoch_2**9_2.json',
 'yolo_model_loss1_batch_size_30_epoch_2**9_2.h5']

In [0]:
%env KERAS_BACKEND=tensorflow
from keras.models import Sequential ,Model
from keras.layers import Dense, Activation, Flatten,Input
from keras.layers import Conv2D, MaxPooling2D, Reshape
from keras.optimizers import SGD, Adam
from keras.layers import LeakyReLU
import numpy as np
import tensorflow as tf

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


In [0]:
xtrain1 = np.load("train_pixel1.5_array_new.npy")
ytrain = np.load('train_label_array_18*32.npy')

ytrain.shape

(2880, 5, 18, 32)

In [0]:
ytrain=ytrain[720:1080]

In [0]:
xtrain1.shape

(360, 720, 1280, 3)

In [0]:
IMAGE_H, IMAGE_W = 720, 1280
grid = 40
GRID_H,  GRID_W = 18, 32
NO_OBJECT_SCALE = 1.0
OBJECT_SCALE = 5.0
COORD_SCALE = 1.0
LOSS_noobj = 0.5
no_batch =  36

In [0]:
def custom_loss(y_true,y_pred):
    true = tf.equal(tf.ones([GRID_H,  GRID_W], tf.float32), 1)
    false = tf.equal(tf.ones([GRID_H,  GRID_W], tf.float32), 0)
    true_mat = tf.tile([tf.concat([tf.tile([true], [3, 1, 1]), tf.tile([false], [2, 1, 1])], 0)], [no_batch,1,1,1])
    xy_pred = tf.where(true_mat, tf.sigmoid(y_pred), tf.zeros([no_batch, 5, GRID_H, GRID_W],tf.float32))
    wh_pred = tf.where(true_mat, tf.zeros([no_batch, 5, GRID_H, GRID_W],tf.float32), tf.exp(y_pred))
    adjusted_y_pred = xy_pred+wh_pred
    true_2 = tf.equal(y_true[:,0], tf.ones([no_batch, GRID_H,  GRID_W], tf.float32))
    no_object_loss = tf.where(true_2, tf.zeros([no_batch,GRID_H,  GRID_W], tf.float32), LOSS_noobj*(y_true[:,0]-adjusted_y_pred[:,0])**2)
    object_loss = tf.where(true_2,(y_true[:,0]-adjusted_y_pred[:,0])**2
                           +OBJECT_SCALE*(y_true[:,1]-adjusted_y_pred[:,1])**2
                           +OBJECT_SCALE*(y_true[:,2]-adjusted_y_pred[:,2])**2
                           +OBJECT_SCALE*(y_true[:,3]**0.5-adjusted_y_pred[:,3]**0.5)**2
                           +OBJECT_SCALE*(y_true[:,4]**0.5-adjusted_y_pred[:,4]**0.5)**2
                           , tf.zeros([no_batch,GRID_H,  GRID_W], tf.float32))
    total_loss = no_object_loss + object_loss
    return total_loss
  

In [0]:
from keras.models import model_from_json
model = model_from_json(open('yolo_model_loss1_batch_size_30_epoch_2**9_2.json').read())
model.load_weights('yolo_model_loss1_batch_size_30_epoch_2**9_2.h5')

In [0]:
model.compile(loss=custom_loss, optimizer=SGD(lr=0.001) )
model.fit(xtrain1,ytrain,validation_split=0.1,batch_size=no_batch, epochs=2**9)

Train on 324 samples, validate on 36 samples
Epoch 1/512
324/324 [==============================] - 31s 96ms/step - loss: 0.0234 - val_loss: 0.0344
Epoch 2/512
324/324 [==============================] - 15s 46ms/step - loss: 0.0222 - val_loss: 0.0310
Epoch 3/512
324/324 [==============================] - 15s 46ms/step - loss: 0.0217 - val_loss: 0.0342
Epoch 4/512
324/324 [==============================] - 15s 46ms/step - loss: 0.0212 - val_loss: 0.0364
Epoch 5/512
324/324 [==============================] - 14s 44ms/step - loss: 0.0201 - val_loss: 0.0316
Epoch 6/512
324/324 [==============================] - 15s 46ms/step - loss: 0.0193 - val_loss: 0.0311
Epoch 7/512
324/324 [==============================] - 15s 46ms/step - loss: 0.0188 - val_loss: 0.0294
Epoch 8/512
324/324 [==============================] - 15s 45ms/step - loss: 0.0182 - val_loss: 0.0333
Epoch 9/512
324/324 [==============================] - 14s 44ms/step - loss: 0.0182 - val_loss: 0.0311
Epoch 10/512
324/324 [======

In [0]:
model_json= model.to_json()
open('yolo_model_loss1_batch_size_36_epoch_2**9_with_cv_3.json','w').write(model_json)
model.save_weights('yolo_model_loss1_batch_size_36_epoch_2**9_with_cv_3.h5')

##新增 cross validation 2

In [0]:
! kill -9 -1

In [0]:
import os
from google.colab import drive
drive.mount('/gdrive')
os.symlink('/gdrive/My Drive', '/content/gdrive')
!ls -l /content/gdrive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
total 12
drwx------ 2 root root 4096 May 20 16:10 'All code and data for Yolo'
drwx------ 2 root root 4096 Jun 13 08:32  Energy
drwx------ 2 root root 4096 May 20 16:12 '中間產生檔案 or 舊檔'


In [0]:
import os
os.chdir('/gdrive/My Drive/All code and data for Yolo/Label and Pixel')
os.listdir()

['train_pixel_firsthalf_array.npy',
 'train_pixel_secondhalf_array.npy',
 'test_label_array_version2.npy',
 'train_label_array_version2.npy',
 'train_label_array.npy',
 'test_label_array.npy',
 'train_label_array_18*32.npy',
 'test_label_array_18*32.npy',
 'train_pixel0.5_array_new.npy',
 'train_pixel1_array_new.npy',
 'train_pixel1.5_array_new.npy',
 'train_pixel2_array_new.npy',
 'yolo_model_loss1_batch_size_40_epoch_15_1.json',
 'yolo_model_loss1_batch_size_40_epoch_15_1.h5',
 'yolo_model_loss1_batch_size_30_epoch_2**9_2.json',
 'yolo_model_loss1_batch_size_30_epoch_2**9_2.h5',
 'yolo_model_loss1_batch_size_36_epoch_2**9_with_cv_3.json',
 'yolo_model_loss1_batch_size_36_epoch_2**9_with_cv_3.h5']

In [0]:
%env KERAS_BACKEND=tensorflow
from keras.models import Sequential ,Model
from keras.layers import Dense, Activation, Flatten,Input
from keras.layers import Conv2D, MaxPooling2D, Reshape
from keras.optimizers import SGD, Adam
from keras.layers import LeakyReLU
import numpy as np
import tensorflow as tf

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


In [0]:
xtrain1 = np.load("train_pixel2_array_new.npy")
ytrain = np.load('train_label_array_18*32.npy')

ytrain.shape

(2880, 5, 18, 32)

In [0]:
ytrain=ytrain[1080:1440]

In [0]:
xtrain1.shape

NameError: ignored

In [0]:
IMAGE_H, IMAGE_W = 720, 1280
grid = 40
GRID_H,  GRID_W = 18, 32
NO_OBJECT_SCALE = 1.0
OBJECT_SCALE = 5.0
COORD_SCALE = 1.0
LOSS_noobj = 0.5
no_batch =  36

In [0]:
def custom_loss(y_true,y_pred):
    true = tf.equal(tf.ones([GRID_H,  GRID_W], tf.float32), 1)
    false = tf.equal(tf.ones([GRID_H,  GRID_W], tf.float32), 0)
    true_mat = tf.tile([tf.concat([tf.tile([true], [3, 1, 1]), tf.tile([false], [2, 1, 1])], 0)], [no_batch,1,1,1])
    xy_pred = tf.where(true_mat, tf.sigmoid(y_pred), tf.zeros([no_batch, 5, GRID_H, GRID_W],tf.float32))
    wh_pred = tf.where(true_mat, tf.zeros([no_batch, 5, GRID_H, GRID_W],tf.float32), tf.exp(y_pred))
    adjusted_y_pred = xy_pred+wh_pred
    true_2 = tf.equal(y_true[:,0], tf.ones([no_batch, GRID_H,  GRID_W], tf.float32))
    no_object_loss = tf.where(true_2, tf.zeros([no_batch,GRID_H,  GRID_W], tf.float32), LOSS_noobj*(y_true[:,0]-adjusted_y_pred[:,0])**2)
    object_loss = tf.where(true_2,(y_true[:,0]-adjusted_y_pred[:,0])**2
                           +OBJECT_SCALE*(y_true[:,1]-adjusted_y_pred[:,1])**2
                           +OBJECT_SCALE*(y_true[:,2]-adjusted_y_pred[:,2])**2
                           +OBJECT_SCALE*(y_true[:,3]**0.5-adjusted_y_pred[:,3]**0.5)**2
                           +OBJECT_SCALE*(y_true[:,4]**0.5-adjusted_y_pred[:,4]**0.5)**2
                           , tf.zeros([no_batch,GRID_H,  GRID_W], tf.float32))
    total_loss = no_object_loss + object_loss
    return total_loss
  

In [0]:
from keras.models import model_from_json
model = model_from_json(open('yolo_model_loss1_batch_size_36_epoch_2**9_with_cv_3.json').read())
model.load_weights('yolo_model_loss1_batch_size_36_epoch_2**9_with_cv_3.h5')


W0804 11:37:07.442998 140665617352576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0804 11:37:07.470747 140665617352576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0804 11:37:07.497454 140665617352576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0804 11:37:07.498348 140665617352576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0804 11:37:07.499312 140665617

In [0]:
model.compile(loss=custom_loss, optimizer=SGD(lr=0.001) )
model.fit(xtrain1,ytrain,validation_split=0.1,batch_size=no_batch, epochs=2**9)

W0804 11:37:23.598582 140665617352576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0804 11:37:23.615009 140665617352576 deprecation.py:323] From <ipython-input-9-e2d6184a2baf>:5: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 324 samples, validate on 36 samples
Epoch 1/512
324/324 [==============================] - 31s 97ms/step - loss: 0.0207 - val_loss: 0.0234
Epoch 2/512
324/324 [==============================] - 13s 41ms/step - loss: 0.0191 - val_loss: 0.0212
Epoch 3/512
324/324 [==============================] - 14s 43ms/step - loss: 0.0182 - val_loss: 0.0202
Epoch 4/512
324/324 [==============================] - 14s 42ms/step - loss: 0.0190 - val_loss: 0.0216
Epoch 5/512
324/324 [==============================] - 14s 42ms/step - loss: 0.0176 - val_loss: 0.0218
Epoch 6/512
324/324 [==============================] - 14s 44ms/step - loss: 0.0172 - val_loss: 0.0220
Epoch 7/512
324/324 [==============================] - 15s 47ms/step - loss: 0.0166 - val_loss: 0.0208
Epoch 8/512
324/324 [==============================] - 15s 47ms/step - loss: 0.0163 - val_loss: 0.0206
Epoch 9/512
324/324 [==============================] - 15s 46ms/step - loss: 0.0165 - val_loss: 0.0218
Epoch 10/512
324/324 [======

In [0]:
model_json= model.to_json()
open('yolo_model_loss1_batch_size_36_epoch_2**9_with_cv_4.json','w').write(model_json)
model.save_weights('yolo_model_loss1_batch_size_36_epoch_2**9_with_cv_4.h5')